# Part-1 ：CI and Null hypothesis significance test (NHST):
### 作业中的x2对应了本脚本中的x3

In [3]:
import numpy as np
import scipy.stats as stats
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

In [3]:
n1=25
np.random.seed(100)
x1=stats.norm.rvs(3,3,n1)+stats.uniform.rvs(-1,1,n1)
x2=stats.f.rvs(2,30,0,1,n1)**2+stats.uniform.rvs(-1,1,n1)
x3=stats.uniform.rvs(-1,1,n1)**2+x1

## （1）请检验x1,x2,x3的正态性；然后根据正态性

In [6]:
print(stats.shapiro(x1)[1])
print(stats.shapiro(x2)[1])
print(stats.shapiro(x3)[1])

0.814449667930603
2.5173179096782405e-07
0.6074895262718201


x2正态性检验p值小于0.05，不满足正态性分布；x1，x3正态性检验p值大于0.05，视为满足正态性分布要求

## （2）计算x1,x2,x3对应总体均值的99% CI

In [8]:
stats.t.interval(0.99, len(x1) - 1, loc=np.mean(x1), scale=stats.sem(x1))

(1.2936857237899495, 4.1922367478347846)

In [12]:
bs.bootstrap(x2, stat_func=bs_stats.mean, alpha=0.01, is_pivotal=False)

3.1733801491355123    (0.26648299882659116, 7.430197473056889)

In [13]:
stats.t.interval(0.99, len(x3) - 1, loc=np.mean(x3), scale=stats.sem(x3))

(1.5525518558575799, 4.4572687891730665)

如上，对x1, x3使用t分布进行区间估计；使用非参数方法bootstrap对x2进行区间估计。

x1：(1.2936857237899495, 4.1922367478347846)

x2：(0.26648299882659116, 7.430197473056889)

x3：(1.5525518558575799, 4.4572687891730665)

## （3）计算x3，x1总体均值差值的95% CI

因题中表意不清，同学们出现了两类答案，1)认为x1, x3互为独立样本；2)因为x3=stats.uniform.rvs(-1,1,n1)**2+x1，将x1，x3视为配对样本。两类回答按照对应方法解答均给分（且与1.5题解法配对）

### 互为独立样本解法 

该方法需要先验证x1，x3的方差齐性，如p>0.05则可视为总体方差相等进行后续计算。

In [15]:
print('variance homogeneity test p-value: {}'.format(stats.levene(x1, x3)[1]))

variance homogeneity test p-value: 0.9292708195609058


方差齐性检验p>0.05,且x1，x3均满足正态分布，所以使用方差未知但相等的公式进行后续计算（公式见课件）ps:区间正负号均取反也正确

In [16]:
alpha = 0.05
n1, n2 = len(x1), len(x3)
var1 = np.var(x1, ddof=1)
var2 = np.var(x3, ddof=1)
sp = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
m = np.mean(x1) - np.mean(x3)
t = stats.t.isf(alpha / 2, n1 + n2 - 2, 0, 1)
moe = sp * t * np.sqrt(1 / n1 + 1 / n2)
ci_low = m - moe
ci_high = m + moe
print('CI of diff between x1 and x3 is [{0},{1}]'.format(ci_low, ci_high))

CI of diff between x1 and x3 is [-1.736901428471208,1.2130032550652963]


### 配对样本解法 

In [17]:
tmp = x1 - x3
stats.t.interval(0.95, len(tmp) - 1, loc=np.mean(tmp), scale=stats.sem(tmp))

(-0.37434272763008009, -0.14955544577583174)

## （4）利用置信区间和NHST两种方法推断x1来自的总体均值是否大于2.0

一般零假设取无期望现象发生，在本题中为H0：x1总体均值小于等于2.0，所得结果为接受零假设。大部分同学取H0为x1总体均值大于2.0，得到对应结果也应为接受零假设。两者结果矛盾，但均为正确，这是由于零假设设定的差异导致的。（下文以H0为x1总体均值小于等于2.0进行演示）

H0:x1总体均值小于等于2.0

置信区间方法：使用单侧置信区间，本题零假设为小于等于，所以使用单侧置信区间下界，即(xxx, + ∞)

NHST方法：使用单边NHST检验
（两种方法其实为同一原理的不同表现方式）

In [18]:
stats.t.interval(0.9, len(x1) - 1, loc=np.mean(x1), scale=stats.sem(x1))

(1.8564424183193413, 3.6294800533053926)

取双边0.9置信区间，通过截取下界即可获得0.95单侧置信区间(1.856, +∞)，单侧置信下界1.856小于2，说明在0.95单侧置信区间内无法说明x1总体均值大于2.0

In [21]:
print(np.mean(x1))
print(stats.ttest_1samp(x1, 2)[1] / 2)

2.74296123581
0.0822606662401


使用单样本双边检验，检验所得p值除以2得到单边p值，p>0.05，接受零假设，无法说明x1来自的总体均值大于2.0

## （5）利用置信区间和NHST两种方法推断x1,x3来自的总体均值是否相等；并计算effect size (Cohen's d)

方法与1.3对应配套，H0：x1，x2来自的总体均值相等

### 互为独立样本解法 

In [24]:
print(stats.ttest_ind(x1, x3)[1])

0.722593051839


经过1.3的方差齐性检验，可将x1，x3视为总体方差相等进行cohen's d计算

In [27]:
def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1) * np.std(x, ddof=1) ** 2 + (ny-1) * np.std(y, ddof=1) ** 2) / dof)

print(cohen_d(x1,x3))

-0.100999001346


由1.3互为独立样本解法可得，x1与x3差值置信区间[-1.736901428471208,1.2130032550652963], 0位于95%置信区间内，接受零假设；t检验结果p值大于0.05，接受零假设；cohen's d为0.1（正负均可）

### 配对样本解法 

In [28]:
print(stats.ttest_rel(x1, x3)[1])

6.7289136941e-05


In [30]:
cohen_d_rel = np.mean(x1 - x3) / np.std(x1 - x3, ddof=1)
print(cohen_d_rel)

-0.962040803744


由1.3配对样本解法，x1与x3差值置信区间[-0.37434272763008009, -0.14955544577583174]，0位于95%置信区间外，拒绝零假设；配对t检验结果p值小于0.05，拒绝零假设；cohen's d为0.96（正负均可）

In [ ]:
# Part-2

Solution
(a) The p-value (0.003) is small so the decision is to reject H0 and conclude that the mean recall 
for sleep (¯xs = 15.25) is different from the mean recall for caffeine (¯xc = 12.25). Since the mean
for the sleep group is higher than the mean for the caffeine group, we have sufficient evidence to
conclude that mean recall after sleep is in fact better than after caffeine. Yes, sleep is really better
for you than caffeine for enhancing recall ability.
(b) The p-value (0.06) is not less than 0.05 so we would not reject H0 at a 5% level, but it is
less than 0.10 so we would reject H0 at a 10% level. There is some moderate evidence of a difference 
in mean recall ability between sleep and a placebo, but not very strong evidence.
(c) The p-value (0.22) is larger than any common significance level, so do not reject H0. The
placebo group had a better mean recall in this sample (¯xp = 13.70 compared to ¯xc = 12.25), but
there is not enough evidence to conclude that the mean for the population would be different for a
placebo than the mean recall for caffeine.
(d) Get a good night’s sleep!


# Part-3 ：ANOVA   睡眠治疗实验

In [16]:
import pingouin as pg
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.anova import AnovaRM

每小题均需要对样本进行正态性和方差齐性检验，由于无论检验结果如何都进行后续anova检验，故在此省略。（缺少正态性及方差齐性检验每小题扣2分）

以下所有小题均可使用同样效果的其他语句替代，所得结果不变

##  （1）表单SleepExp_1.csv 是招募60名被试，随机分成三种不同剂量组（10mg, 50mg, 100mg)进行试验 ，表单Scores是治疗后被试的评分，请推断不同剂量组间是否有治疗效果差异？

In [11]:
df = pd.read_csv('./samples/SleepExp_1.csv')
df.head()

,subjects,Dosage,Scores
0,S1,10mg,73.686522
1,S2,10mg,70.611993
2,S3,10mg,84.661216
3,S4,10mg,59.702051
4,S5,10mg,69.836893


In [14]:
anova_lm(ols('Scores~C(Dosage)',data=df).fit())

,df,sum_sq,mean_sq,F,PR(>F)
C(Dosage),2.0,3438.030406,1719.015203,10.480888,0.000133
Residual,57.0,9348.813276,164.014268,NaN,NaN


使用单样本ANOVA检验，p值小于0.05，说明剂量对治疗评分有显著影响

## （2）表单SleepExp_2.csv是招募20名被试，每个被试连续进行了三种剂量治疗的（10mg, 50mg, 100mg)实验 ，表单Scores是每个剂量治疗后被试的评分，请推断不同剂量组间是否有治疗效果差异？

In [15]:
df = pd.read_csv('./samples/SleepExp_2.csv')
df.head()

,Subjects,Dosage,Scores
0,S1,10mg,59.497332
1,S2,10mg,95.574353
2,S3,10mg,79.346071
3,S4,10mg,65.558919
4,S5,10mg,68.947775


In [17]:
aovrm = AnovaRM(df, 'Scores', 'Subjects', within=['Dosage'])
res = aovrm.fit()

print(res)

               Anova
       F Value Num DF  Den DF Pr > F
------------------------------------
Dosage  2.4209 2.0000 38.0000 0.1024



使用非独立样本ANOVA检验，p值大于0.05，说明剂量对治疗评分影响不显著

##  （3）表单SleepExp_3.csv是招募了30名被试，每个被试连续进行了三种剂量治疗的（10mg, 50mg, 100mg)实验 ，表单Scores是每个剂量治疗后被试的评分，请推断剂量、性别、及剂量与性别相互作用的效应分别对治疗评分的影响是否显著？

In [18]:
df = pd.read_csv('./samples/SleepExp_3.csv')
df.head()

,Subjects,Dosage,Scores,Gender
0,S1,10mg,73.307958,Female
1,S2,10mg,80.138753,Female
2,S3,10mg,75.630965,Female
3,S4,10mg,77.649084,Female
4,S5,10mg,84.920238,Female


In [19]:
pg.mixed_anova(data=df, dv='Scores', within='Dosage', between='Gender', subject='Subjects')

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,Gender,84.627,1,28,84.627,2.178,1.511432e-01,0.072,-
1,Dosage,931.005,2,56,465.502,19.115,4.699411e-07,0.406,0.957
2,Interaction,47.620,2,56,23.810,0.978,3.825007e-01,0.034,-


使用混合ANOVA检验，注意within='Dosage', between='Gender'，所得结果p值说明剂量对治疗得分有显著影响，性别/性别与剂量的相互作用对治疗得分没有显著影响。

##  （4）表单SleepExp_4.csv是招募了15名被试，每个被试分别在春季，秋季都连续进行了三种剂量治疗的（10mg, 50mg, 100mg)实验 ，表单Scores是每个剂量治疗后被试的评分，请推断剂量、季节、及剂量与季节相互作用的效应分别对治疗评分的影响是否显著？

In [21]:
df = pd.read_csv('./samples/SleepExp_4.csv')
df.head()

,Subjects,Dosage,Scores,Season
0,S1,10mg,70.542837,Spring
1,S2,10mg,78.720910,Spring
2,S3,10mg,81.931458,Spring
3,S4,10mg,70.631059,Spring
4,S5,10mg,85.274465,Spring


In [22]:
pg.rm_anova(data=df, dv='Scores', subject='Subjects', within=['Dosage', 'Season'])

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,np2,eps
0,Dosage,2400.691,2,28,1200.346,36.566,1.555154e-08,3.795746e-07,0.723,0.788
1,Season,368.954,1,14,368.954,6.738,2.115423e-02,2.115423e-02,0.325,1.000
2,Dosage * Season,78.718,2,28,39.359,0.829,4.468987e-01,4.273312e-01,0.056,0.818


使用非独立样本ANOVA检验，注意within=['Dosage', 'Season']，所得结果p值说明剂量/季节对治疗得分有显著影响，但是剂量与季节的相互作用对治疗得分没有显著影响

##  （5）表单SleepExp_5.csv从上海、北京招募了90名被试，随机分成三种剂量治疗组（10mg, 50mg, 100mg) 进行睡眠实验，表单Scores是每个被试治疗后的评分，请推断剂量、城市、及剂量与城市相互作用的效应分别对治疗评分的影响是否显著？

In [23]:
df = pd.read_csv('./samples/SleepExp_5.csv')
df.head()

,Subjects,Dosage,Scores,City
0,S1,10mg,78.686760,Shanghai
1,S2,10mg,71.913514,Shanghai
2,S3,10mg,77.314652,Shanghai
3,S4,10mg,73.279049,Shanghai
4,S5,10mg,79.897818,Shanghai


In [24]:
pg.anova(data=df, dv='Scores', between=['Dosage', 'City'])

,Source,SS,DF,MS,F,p-unc,np2
0,Dosage,3820.314,2,1910.157,90.684590,1.042341e-21,0.683460
1,City,16.548,1,16.548,0.785615,3.779598e-01,0.009266
2,Dosage * City,292.849,2,146.425,6.951497,1.608049e-03,0.142008
3,Residual,1769.355,84,21.064,NaN,NaN,NaN


使用独立样本ANOVA检验，注意between=['Dosage', 'City']，所得结果p值说明剂量/剂量与城市的相互作用对治疗得分有显著影响，城市对治疗得分影响不显著